In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# pd.set_option('display.max_rows',500)
df_admi = pd.read_csv('./dataset/ADMISSIONS.csv')
df_diag = pd.read_csv('./dataset/DIAGNOSES_ICD.csv')
df_icu = pd.read_csv('./dataset/ICUSTAYS.csv')
df_patient = pd.read_csv('./dataset/PATIENTS.csv')
df_ditems = pd.read_csv('./dataset/D_ITEMS.csv')
# df_charts = pd.read_csv('./dataset/CHARTEVENTS.csv')

# df_admi.head(3)
# df_diag.head(3)
# df_icu.head(3)
# df_patient.head(3)
# df_ditems.head(3)
# df_charts.head(3)

In [22]:
# 1. find all patients with sepsis
df_sepsis = df_diag.loc[df_diag['ICD9_CODE'] == '99591']
df_sepsis.head(3)


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
276,505,64,172056,3.0,99591
450,679,85,112077,18.0,99591
1084,477,61,189535,8.0,99591


In [23]:
# 2. icu stays bigger then 24 hours
df_los = df_icu.loc[df_icu['LOS'] >= 24]
df_los.head(3)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
14,379,283,109185,231490,carevue,MICU,MICU,15,15,2166-08-12 22:03:26,2166-09-12 14:41:42,30.6932
17,382,285,165312,238023,carevue,TSICU,TSICU,14,14,2152-09-21 22:48:50,2152-10-20 15:19:31,28.6880
18,383,286,106909,260225,carevue,MICU,MICU,52,52,2175-12-31 22:57:27,2176-02-24 19:16:58,54.8469


In [51]:
# 3. vital signs --> take mean
df_vital = df_ditem.loc[df_ditem['LABEL'].str.contains("Heart", na=False)]
df_vital 
# EtCO2(228640)
# Hear Rate(220045)
# hearrate_min(3494)

# temperature Celsius(223762) --> min/max
# Artierial Blood Pressure systolic --> ABPs(220050)
# Artierial Blood Pressure diastolic --> ABPd(220051)
# Artierial Blood Pressure mean --> ABPm(220052)


# Respiratory Rate(220210)

# sp2--> o2 saturation pulseoxymetry(220277)



,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
475,212,211,Heart Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
476,213,212,Heart Rhythm,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
477,214,213,Heart Sounds,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1897,1078,3494,Lowest Heart Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10742,13322,224389,Heart Sounds,Heart Sounds,metavision,chartevents,Cardiovascular,NaN,Text,NaN
11424,15197,228198,Tandem Heart Flow,Tandem Heart Flow,metavision,chartevents,Tandem Heart,L/min,Numeric,NaN
11425,15198,228199,Type (Tandem Heart),Type (Tandem Heart),metavision,chartevents,Tandem Heart,NaN,Text,NaN
11427,15200,228201,Tandem Heart Access Line,Tandem Heart Access Line,metavision,procedureevents_mv,Access Lines - Invasive,None,Process,NaN
11428,15201,228202,Tandem Heart Return Line,Tandem Heart Return Line,metavision,procedureevents_mv,Access Lines - Invasive,None,Process,NaN
11429,15202,228203,Tandem Heart Access Line Cannula Site,Tandem Heart Access Line Cannula Site,metavision,chartevents,Access Lines - Invasive,NaN,Text,NaN


In [ ]:
# 4. merge tables
